Test persoon: 621e346f67b776a24081744f

In [ ]:
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
client = MongoClient("mongodb://localhost:27017/")
	
db = client["lifesnaps"]
collection = db["fitbit"]

person = "621e2f9167b776a240011ccb"

query = {
    "id": ObjectId(person),
    "type": "sleep"
}

doc = collection.find(query)

In [ ]:
records = []

for doc in collection.find(query):
    try:
        # Totaal aantal slaap in 1 nacht:
        total_sleep = doc["data"]["minutesAsleep"]/60
        # Deep, light en rem slaap in 1 nacht: 
        deep = doc["data"]["levels"]["summary"]["deep"]["minutes"] / 60
        light = doc["data"]["levels"]["summary"]["light"]["minutes"] / 60
        rem = doc["data"]["levels"]["summary"]["rem"]["minutes"] / 60
        
        date = pd.to_datetime(doc["data"]["dateOfSleep"])
        records.append({"date": date, "total_sleep": total_sleep, "light": light, "deep": deep,"rem": rem})
    except:
        print("foutje")
    
df = pd.DataFrame(records)
df.set_index('date', inplace=True)
df = df.sort_index()


In [ ]:
df

In [ ]:
# Berekent het gemiddelde aantal uren slaap van de afgelopen 7 dagen
# Start wel pas vanaf dag 8
weekly_avg = df['total_sleep'].rolling(window=7).mean()
df['weekly_avg'] = weekly_avg.shift(1)
print(df[['total_sleep','weekly_avg']])

In [ ]:
# Voorbeeld van hoe je een specifieke maand kan selecteren:
df_test = df.loc['2021-11']
print(df_test)

In [ ]:
# Grafiek van aantal uren slaap van 1 maand
plt.figure()
plt.plot(df_test.index, df_test['total_sleep'], marker='o', linestyle='-')
plt.title("Daily Total Sleep Hours")
plt.xlabel("Date")
plt.ylabel("Hours of Sleep")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 1 specifieke dag zoeken: 
df_day = df.loc['2021-12-15']
print(df_day)

In [ ]:
df.to_csv("csv/Sleep.csv")